In [1]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

import io
pre_covid_df = pd.read_csv(io.BytesIO(uploaded['Project_Data_Pre-Covid.csv']))

Saving Project_Data_Pre-Covid.csv to Project_Data_Pre-Covid.csv


In [2]:
pre_covid_df.head()

,product_category,product_url,review_date,Date,handle,rating,helpfulness_rating,review
0,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,4/28/2019,Pre_Covid,Wrich,5,NaN,As described. good value!
1,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,5/21/2019,Pre_Covid,Amy H.,5,NaN,awesome for the price. fit nice and dont rip s...
2,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,6/19/2019,Pre_Covid,Frankie Boy,2,3,"Every time i used for washing, there’s a moist..."
3,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,6/21/2019,Pre_Covid,Frances Parish,5,NaN,Exactly the right size. Comfortable.
4,disposable_gloves,https://www.amazon.com/dp/B07KYV178H,5/21/2019,Pre_Covid,Stan,5,NaN,Exactly what needed and described


In [3]:
pre_covid_df = pre_covid_df.drop(columns=['product_url', 'helpfulness_rating'], axis=1)

In [4]:
pre_covid_df['rating'] = pre_covid_df['rating'].apply(lambda z:'positive' if z>3 else 'negative')

In [5]:
pre_covid_df.head()

,product_category,review_date,Date,handle,rating,review
0,disposable_gloves,4/28/2019,Pre_Covid,Wrich,positive,As described. good value!
1,disposable_gloves,5/21/2019,Pre_Covid,Amy H.,positive,awesome for the price. fit nice and dont rip s...
2,disposable_gloves,6/19/2019,Pre_Covid,Frankie Boy,negative,"Every time i used for washing, there’s a moist..."
3,disposable_gloves,6/21/2019,Pre_Covid,Frances Parish,positive,Exactly the right size. Comfortable.
4,disposable_gloves,5/21/2019,Pre_Covid,Stan,positive,Exactly what needed and described


In [6]:
import nltk

nltk.download('punkt')

import string
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
pre_covid_df['review'] = pre_covid_df['review'].astype(str)

In [8]:
from nltk.tokenize import word_tokenize

for item in pre_covid_df['review']:
  words = word_tokenize(item.replace('\n',' '))

In [9]:
clean = [word for word in words if word not in set(string.punctuation)]

In [10]:
characters_to_remove = ["''",'``','!',"'"]
clean_words = [word for word in clean if word not in set(characters_to_remove)]

In [11]:
nltk.download('stopwords')

from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
clean = [word for word in clean_words if word not in english_stops]

In [13]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [14]:
reviews = [(list(clean), rating)
              for review_text in pre_covid_df['review']
              for rating in pre_covid_df['rating']]

random.shuffle(reviews)

In [15]:
clean = [wordnet_lemmatizer.lemmatize(word) for word in clean]

In [16]:
all_words = nltk.FreqDist(word.lower() for word in clean)
word_features = list(all_words)

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [17]:
split = 0.7

featuresets = [(document_features(d), c) for (d,c) in reviews]

train_set, test_set = featuresets[int(round(len(featuresets)*split,0))-1:], featuresets[:int(round(len(featuresets)*split,0))-1]

nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [18]:
print(nltk.classify.accuracy(nb_classifier, test_set))

0.8012718600953895


In [19]:
nb_classifier.show_most_informative_features(n=5)

Most Informative Features
           contains(buy) = True           negati : positi =      1.0 : 1.0
         contains(would) = False          negati : positi =      1.0 : 1.0
